In [2]:
import pandas as pd
import datetime as dt
from datetime import date
import numpy as np
from scipy import stats
import seaborn as sns

In [3]:
df = pd.read_csv('Detect4_1-anon.csv')
df2 = pd.read_csv('Detect4_2-anon.csv')

combined =[df, df2]
table = pd.concat(combined, axis = 1)
data = table.loc[:,~table.columns.duplicated()] #both spreads combined, with MONTH and ID removed

In [4]:
#three dataframes for baseline, 3-month, and 24-month
#96 have baseline, 95 have 3-month, and 85 have 24-month
baselist = []
threemonth = []
twentyfour = []
dupcount = []

data = data[data['VISIT_DATE'].notna()] #Removes entries that did not have a recorded visit date 
dupcount = data.pivot_table(index = 'ID', aggfunc = 'size') #Finds how many instances of IDs 
invalid3 = []
dupcount = pd.DataFrame(data = dupcount)

for index, val in dupcount.iterrows():
    if dupcount.at[index, 0] < 3:
        invalid3.append(index)
invalidid = pd.DataFrame(data = invalid3)
invalidid #List of Invalid IDs

data = data[~data.ID.isin(invalid3)] #Removes invalid ids, only patients with all three visits are kept

for index, row in data.iterrows():
    if data.at[index, 'MONTH'] == 0:
        baselist.append(row)
    if data.at[index, 'MONTH'] == 3:
        threemonth.append(row)
    if data.at[index, 'MONTH'] == 24:
        twentyfour.append(row)
baseline = pd.DataFrame(data = baselist)
three = pd.DataFrame(data = threemonth)
twentyfour1 = pd.DataFrame(data = twentyfour)

#write data to csv
data.to_csv('cleaned-data.csv')

In [5]:
#TABLE 1 DATA

#height
height = "{} in +- {}"
roundedheight = round(data['HEIGHT_IN'].mean(), 2)
roundedstd = round(data['HEIGHT_IN'].std(),1)
height1 = height.format(roundedheight,roundedstd)


#weight
weight = " {} lbs +- {}"
roundedweight = round(data['WEIGHT_LB'].mean(),2)
roundedstdw = round(data['WEIGHT_LB'].std(),1)
weight1 = weight.format(roundedweight,roundedstdw)


#gender
female = "Female:{} ({})"
male = "Male: {} ({})"
gen = "Male: {} ({}) Female: {} ({})"
femalecount =0
malecount =0
for index, row in data.iterrows(): 
    if data.at[index, 'MONTH'] == 0:
        if data.at[index,'gender']== 2:
            #print('female')
            femalecount += 1
        elif data.at[index, 'gender']==1:
            #print('male')
            malecount += 1
totalsex = femalecount + malecount
femaleperc = round((femalecount / totalsex)*100,1)
maleperc = round((malecount / totalsex)*100,1)
gender = gen.format(malecount, maleperc, femalecount, femaleperc)


#race
Race = "  Black: {} ({})  White: {} ({})" 
Race2 = "Race: \n Black:   \n White: "

Black = "Black:{} ({})"
White = "White:{} ({})"
blackcount = 0
whitecount =0
for index, row in data.iterrows():
    if data.at[index, 'MONTH'] ==0:
        if data.at[index, 'RACE']==3:
            blackcount += 1
        elif data.at[index,'RACE']==5:
            whitecount += 1
totalrace = blackcount + whitecount 
blackperc = round((blackcount / totalrace)*100,1)
whiteperc = round((whitecount/totalrace)*100,1)
black1 = Black.format(blackcount, blackperc)
white1 = White.format(whitecount,whiteperc)
race1 = Race.format(blackcount, blackperc, whitecount, whiteperc)



#age
Age = " {} +- {}"
age = 0
now = pd.Timestamp('now')
data['DOB'] = pd.to_datetime(data['DOB']) 
data['DOB'] = data['DOB'].where(data['DOB'] < now, data['DOB'] -  np.timedelta64(100, 'Y'))   # 2
data['age'] = (now - data['DOB']).astype('<m8[Y]')    # 3
roundedavgage = round(data['age'].mean(),1)
roundedstdage = round(data['age'].std(),1)
age1 = Age.format(roundedavgage, roundedstdage)



#hypertension at baseline?
hpt = "{} ({})"
hptcount =0
for index, row in data.iterrows(): 
    if data.at[index, 'MONTH']==0:
        if data.at[index,'HTNNEW']== 1:
        #print('female')
            hptcount += 1
totalpt = 71
htpperc = round((hptcount / totalpt)*100,1)
hptval = hpt.format(hptcount, htpperc)

#type of cancer treatment
chemo= "Anthracycline:{} ({})  Trastuzumab: {} ({})  Taxane: {} ({}) Cyclophosphamide: {} ({}) Other: {} ({})"
anthcount = 0
trast = 0
tax = 0
cyc = 0
other = 0
for index, row in data.iterrows():
    if data.at[index, 'MONTH']== 0:
        if data.at[index, 'ANTHRACYCLINE']==1:
            anthcount += 1
        if data.at[index, 'TRASTUZUMAB']==1:
            trast += 1
        if data.at[index, 'TAXANE']==1:
            tax += 1
        if data.at[index, 'CYCLO']==1:
            cyc += 1
        if data.at[index, 'OTHERCHEMO']==1:
            other += 1
totalpt = 71
anthp = round((anthcount / totalpt)*100,1)
trastp = round((trast / totalpt)*100,1)
taxp = round((tax / totalpt)*100,1)
cycp = round((cyc / totalpt)*100,1)
otherp = round((other / totalpt)*100,1)
chemoval = chemo.format(anthcount, anthp, trast, trastp, tax, taxp, cyc, cycp, other, otherp)

#bmi
bmi30 = 0
notbmi = 0
bmival = "{} ({})"
for index, row in data.iterrows():
    if data.at[index, 'MONTH'] == 0:
        if data.at[index, 'BMI']>= 30:
            bmi30 += 1
totalpt = 71
bmiperc = round((bmi30 / totalpt)*100 , 1)
bmi = bmival.format(bmi30, bmiperc)


#cardioprotective meds
aceinhibcount = 0
diurcount = 0
betacount = 0
angcount = 0
totalpt = 71
CARDMED = "Ace Inhibitors: {}% Diuretics: {}% BetaBlockers: {}% Angiotensin Blockers: {}%"

for index, row in data.iterrows():
    if pd.isnull(data.at[index, 'ACE_INHIB_YR'])== False:
        aceinhibcount += 1
    if pd.isnull(data.at[index, 'DIURETICS_YR'])== False:
        diurcount += 1
    if pd.isnull(data.at[index, 'BETABLOCK_YR'])== False:
        betacount += 1
    if pd.isnull(data.at[index, 'ANGIOTENS_YR']) == False:
        angcount += 1
aceinhibperc = round((aceinhibcount / totalpt)*100,1)
diurperc = round((diurcount/ totalpt)*100,1)
betaperc = round((betacount / totalpt)*100,1)
angperc = round((angcount / totalpt)*100,1)
CARDIO = CARDMED.format(aceinhibperc, diurperc, betaperc, angperc)


In [6]:
npr = np.array([['Age', age1],['Height', height1],['Weight', weight1], ['Race' , race1],['Gender', gender], ['Hypertension',hptval], ['Treatment Regimen', chemoval],['BMI ', bmi], ['Cardioprotective Medications', CARDIO]])
table1 = pd.DataFrame(data = npr, columns = ["Characteristics", "Result"] )

table1

,Characteristics,Result
0,Age,59.8 +- 14.3
1,Height,66.21 in +- 3.9
2,Weight,185.18 lbs +- 44.0
3,Race,Black: 17 (20.0) White: 68 (80.0)
4,Gender,Male: 30 (35.3) Female: 55 (64.7)
5,Hypertension,46 (64.8)
6,Treatment Regimen,Anthracycline:8 (11.3) Trastuzumab: 0 (0.0) ...
7,BMI,36 (50.7)
8,Cardioprotective Medications,Ace Inhibitors: 23.9% Diuretics: 32.4% BetaBlo...


In [5]:
#TABLE 2 DATA 

#Average heart rate at baseline, threemonth, and 24month, and std
bmean = round(baseline['hr_mesa'].mean(), 2)
bstd = round(baseline['hr_mesa'].std(),2)
msb = '{} +- {}'
meanstdbaseline = msb.format(bmean, bstd)

thmean = round(three['hr_mesa'].mean(), 2)
thstd = round(three['hr_mesa'].std(),2)
#meanstdbaseline = str(bmean) + '+-' + str(bstd)
tsb = '{} +- {}'
meanstdthr = tsb.format(thmean, thstd)

twmean = round(twentyfour1['hr_mesa'].mean(), 2)
twstd = round(twentyfour1['hr_mesa'].std(),2)
#meanstdbaseline = str(bmean) + '+-' + str(bstd)
twsb = '{} +- {}'
meanstdtw = twsb.format(twmean, twstd)

hr_3month, hr_3monthp = stats.ttest_rel(baseline['hr_mesa'], three['hr_mesa']) 
hr_24month, hr_24monthp = stats.ttest_rel(baseline['hr_mesa'], twentyfour1['hr_mesa'])

#systolic blood pressure 
sbpmean = round(baseline['sys_bp_mesa'].mean(), 2)
sbpstd = round(baseline['sys_bp_mesa'].std())
msbp = '{} +- {}'
sbpbase = msbp.format(sbpmean, sbpstd)

sbpmeanth = round(three['sys_bp_mesa'].mean(), 2)
sbpstdth = round(three['sys_bp_mesa'].std())
thsbp = '{} +- {}'
sbpth = thsbp.format(sbpmeanth, sbpstdth)

sbpmeantw = round(twentyfour1['sys_bp_mesa'].mean(), 2)
sbpstdtw = round(twentyfour1['sys_bp_mesa'].std())
twsbp = '{} +- {}'
sbptw = twsbp.format(sbpmeantw, sbpstdtw)

sbp_3month, sbp_3monthp = stats.ttest_rel(baseline['sys_bp_mesa'], three['sys_bp_mesa'])
sbp_24month, sbp_24monthp = stats.ttest_rel(baseline['sys_bp_mesa'], twentyfour1['sys_bp_mesa'])

#diastolic blood pressure
dbpmean = round(baseline['dia_bp_mesa'].mean(), 2)
dbpstd = round(baseline['dia_bp_mesa'].std())
dsbp = '{} +- {}'
dbpbase = dsbp.format(dbpmean, dbpstd)

dbpmeanth = round(three['dia_bp_mesa'].mean(), 2)
dbpstdth = round(three['dia_bp_mesa'].std())
thdbp = '{} +- {}'
dbpth = thdbp.format(dbpmeanth, dbpstdth)

dbpmeantw = round(twentyfour1['dia_bp_mesa'].mean(), 2)
dbpstdtw = round(twentyfour1['dia_bp_mesa'].std())
twdbp = '{} +- {}'
dbptw = twdbp.format(dbpmeantw, dbpstdtw)

dbp_3month, dbp_3monthp = stats.ttest_rel(baseline['dia_bp_mesa'], three['dia_bp_mesa'])
dbp_24month, dbp_24monthp = stats.ttest_rel(baseline['dia_bp_mesa'], twentyfour1['dia_bp_mesa'])

#LVEDV
edvmean = round(baseline['vol5_lvedv'].mean(), 2)
edvstd = round(baseline['vol5_lvedv'].std())
edv = '{} +- {}'
edvbase = edv.format(edvmean, edvstd)

edvmeanth = round(three['vol5_lvedv'].mean(), 2)
edvstdth = round(three['vol5_lvedv'].std())
thedv = '{} +- {}'
edvth = thedv.format(edvmeanth, edvstdth)

edvmeantw = round(twentyfour1['vol5_lvedv'].mean(), 2)
edvstdtw = round(twentyfour1['vol5_lvedv'].std())
twedv = '{} +- {}'
edvtw = twedv.format(edvmeantw, edvstdtw)
edv_3month, edv_3monthp = stats.ttest_rel(baseline['vol5_lvedv'], three['vol5_lvedv'])
edv_24month, edv_24monthp = stats.ttest_rel(baseline['vol5_lvedv'], twentyfour1['vol5_lvedv'])

#LVESV
esvmean = round(baseline['vol5_lvesv'].mean(), 2)
esvstd = round(baseline['vol5_lvesv'].std())
esv = '{} +- {}'
esvbase = esv.format(esvmean, esvstd)

esvmeanth = round(three['vol5_lvesv'].mean(), 2)
esvstdth = round(three['vol5_lvesv'].std())
thesv = '{} +- {}'
esvth = thesv.format(esvmeanth, esvstdth)

esvmeantw = round(twentyfour1['vol5_lvesv'].mean(), 2)
esvstdtw = round(twentyfour1['vol5_lvesv'].std())
twesv = '{} +- {}'
esvtw = twesv.format(esvmeantw, esvstdtw)

esv_3month, esv_3monthp = stats.ttest_rel(baseline['vol5_lvesv'], three['vol5_lvesv'])
esv_24month, esv_24monthp = stats.ttest_rel(baseline['vol5_lvesv'], twentyfour1['vol5_lvesv'])

#SV
svmean = round(baseline['VOL5_SV'].mean(), 2)
svstd = round(baseline['VOL5_SV'].std())
sv = '{} +- {}'
svbase = sv.format(svmean, svstd)

svmeanth = round(three['VOL5_SV'].mean(), 2)
svstdth = round(three['VOL5_SV'].std())
thsv = '{} +- {}'
svth = thsv.format(svmeanth, svstdth)

svmeantw = round(twentyfour1['VOL5_SV'].mean(), 2)
svstdtw = round(twentyfour1['VOL5_SV'].std())
twsv = '{} +- {}'
svtw = twsv.format(svmeantw, svstdtw)

sv_3month, sv_3monthp = stats.ttest_rel(baseline['VOL5_SV'], three['VOL5_SV'])
sv_24month, sv_24monthp = stats.ttest_rel(baseline['VOL5_SV'], twentyfour1['VOL5_SV'])

#EF
efmean = round(baseline['vol5_lvef'].mean(), 2)
efstd = round(baseline['vol5_lvef'].std())
ef = '{} +- {}'
efbase = ef.format(efmean, efstd)

efmeanth = round(three['vol5_lvef'].mean(), 2)
efstdth = round(three['vol5_lvef'].std())
thef = '{} +- {}'
efth = thef.format(efmeanth, efstdth)

efmeantw = round(twentyfour1['vol5_lvef'].mean(), 2)
efstdtw = round(twentyfour1['vol5_lvef'].std())
twef = '{} +- {}'
eftw = twef.format(efmeantw, efstdtw)

ef_3month, ef_3monthp = stats.ttest_rel(baseline['vol5_lvef'], three['vol5_lvef'])
ef_24month, ef_24monthp = stats.ttest_rel(baseline['vol5_lvef'], twentyfour1['vol5_lvef'])

#LVMASS
msmean = round(baseline['vol5_lvmass'].mean(), 2)
msstd = round(baseline['vol5_lvmass'].std())
ms = '{} +- {}'
msbase = ms.format(msmean, msstd)

msmeanth = round(three['vol5_lvmass'].mean(), 2)
msstdth = round(three['vol5_lvmass'].std())
thms = '{} +- {}'
msth = thms.format(msmeanth, msstdth)

msmeantw = round(twentyfour1['vol5_lvmass'].mean(), 2)
msstdtw = round(twentyfour1['vol5_lvmass'].std())
twms = '{} +- {}'
mstw = twms.format(msmeantw, msstdtw)

ms_3month, ms_3monthp = stats.ttest_rel(baseline['vol5_lvmass'], three['vol5_lvmass'])
ms_24month, ms_24monthp = stats.ttest_rel(baseline['vol5_lvmass'], twentyfour1['vol5_lvmass'])


In [8]:
table2 = np.array([[meanstdbaseline,meanstdthr, meanstdtw, hr_3monthp, hr_24monthp], [sbpbase, sbpth, sbptw, sbp_3monthp, sbp_24monthp], [dbpbase, dbpth, dbptw, dbp_3monthp, dbp_24monthp], 
                   [edvbase, edvth, edvtw, edv_3monthp, edv_24monthp], [esvbase, esvth, esvtw, esv_3monthp, esv_24monthp], [svbase, svth, svtw,sv_3monthp, sv_24monthp],
                  [efbase, efth, eftw, ef_3monthp, ef_24monthp], [msbase, msth, mstw, ms_3monthp, ms_24monthp]])

table2 = pd.DataFrame(data = table2, index = ["Heart Rate", "Systolic BP", "Diastolic BP", "LVEDV", "LVESV", "Stroke Volume", 
                                                "Ejection Fraction", "LV Mass"], columns = ["Baseline", "3 Months", "24 Months", "3-Month Change", "24-Month Change"])       


meantable = np.array([[bmean, sbpmean, dbpmean, edvmean, esvmean, svmean, efmean, msmean],[thmean, sbpmeanth, dbpmeanth, edvmeanth, esvmeanth, svmeanth, efmeanth, msmeanth], 
                    [twmean, sbpmeantw, dbpmeantw, edvmeantw, esvmeantw, svmeantw, efmeantw, msmeantw]])
meandf = pd.DataFrame(data = meantable, index = ["Baseline", "3 Months", "24 Months"], columns = ["Heart Rate", "Systolic BP", "Diastolic BP", "LVEDV", "LVESV", "Stroke Volume", 
                                                "Ejection Fraction", "LV Mass"]) 
table2


,Baseline,3 Months,24 Months,3-Month Change,24-Month Change
Heart Rate,73.27 +- 15.07,81.0 +- 11.91,70.69 +- 10.48,nan,nan
Systolic BP,116.77 +- 16,108.65 +- 14,115.06 +- 18,nan,nan
Diastolic BP,69.09 +- 12,64.92 +- 11,68.39 +- 12,nan,nan
LVEDV,125.51 +- 35,121.58 +- 36,123.35 +- 41,0.08634122735805144,nan
LVESV,48.9 +- 21,50.41 +- 20,52.5 +- 24,0.24231497235273577,nan
Stroke Volume,76.61 +- 18,71.17 +- 19,70.84 +- 23,0.0009350301309886498,nan
Ejection Fraction,62.0 +- 8,59.23 +- 7,58.3 +- 9,0.000319322263649384,nan
LV Mass,105.85 +- 27,105.12 +- 25,102.88 +- 28,0.6307260894974756,nan


In [9]:
#CHANGE IN LVESV 
#ATTEMPT TO DO FIGURE 2 
val = []
val2= []
val3 = []
edvch = []
edvch2 = []
lvefb = []
lvef3 = []
for index, row in baseline.iterrows():
    val.append(baseline.loc[index,  'vol5_lvesv'])
    edvch.append(baseline.loc[index, 'vol5_lvedv'])
    lvefb.append(baseline.loc[index, 'vol5_lvef'])
val1 = pd.DataFrame(data = val)
edvch = pd.DataFrame(data = edvch)
lvefb = pd.DataFrame(data = lvefb)

for index, row in three.iterrows():
    val2.append(three.loc[index, 'vol5_lvesv'])
    edvch2.append(three.loc[index, 'vol5_lvedv'])
    lvef3.append(three.loc[index, 'vol5_lvef'])
val3 = pd.DataFrame(data = val2)
edvch2 = pd.DataFrame(data = edvch2)
chlvesv = np.subtract(val3, val1)
chlvedv = np.subtract(edvch2, edvch)
lvef3 = pd.DataFrame(data = lvef3)
changes = [chlvesv, chlvedv]


#lvef decline
g1 = []
g2 = []
lvefch = np.subtract(lvef3, lvefb)

lvefperc = lvefch / lvefb
lvefperc = np.array(lvefperc)
for val in lvefperc:
    if val < 0.05:
        g1.append(val)
    elif val >= 0.05:
        g2.append(val)
g1 = pd.DataFrame(data = g1, columns = ['Less than 5% Decline'])
g2 = pd.DataFrame(data = g2, columns = ['Greater than 5% Decline'])

lvef = [g1, g2]
lvefdec = pd.concat(lvef, axis = 1)
lvefdec #sorted perc declines in two groups

,Less than 5% Decline,Greater than 5% Decline
0,-0.098584,0.096662
1,-0.075146,0.095321
2,0.049305,0.058950
3,0.046753,0.067457
4,-0.013346,0.065050
...,...,...
60,-0.027523,NaN
61,-0.093768,NaN
62,-0.282636,NaN
63,-0.104072,NaN
